In [4]:
import yaml
import wandb
import torchreid

config_path = "configs/im_osnet_ain_x1_0_softmax_256x128_amsgrad_cosine.yaml"
with open(config_path, 'r') as file:
    config = yaml.safe_load(file)

# wandb.init(project=config['wandb']['project'],
#            entity=config['wandb']['entity'],
#            config=config)


In [1]:
import torchreid
model_name = 'resnet50'
dataset_name = 'market1501'
datamanager = torchreid.data.ImageDataManager(
    root            = config['data']['root'],
    sources         = config['data']['sources'],
    targets         = config['data']['targets'],
    height          = config['data']['height'],
    width           = config['data']['width'],
    batch_size_train= config['data']['batch_size'],
    batch_size_test = config['data']['batch_size'],
    transforms      = config['data']['transforms']
)

/home/trewq/Desktop/Projects/RetailAnalytics/REID/RESEARCH/deep-person-reid/torchreid/metrics/rank.py:11: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  warnings.warn(


Building train transforms ...
+ resize to 256x128
+ random flip
+ random crop (enlarge to 288x144 and crop 256x128)
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------
=> Loading test (target) dataset
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gal

In [2]:
train_loader = datamanager.train_loader
for batch_idx, data in enumerate(train_loader):
    print(data['pid'])
    break

tensor([131, 201, 386, 436, 377, 132, 374, 375, 212, 747, 484, 428, 329, 203,
        346, 256, 701,  25, 302, 748, 469, 544, 638, 704, 101,  44, 248, 361,
        494, 401, 591, 402, 663, 209, 147,  11,  73, 370, 259, 239, 320, 416,
        135, 675, 678, 668, 290, 407, 414, 701, 224, 567, 249, 176, 688, 586,
        378, 694,  45, 675,   6,  59,  54,   6, 140, 468,  11, 520, 468, 224,
        282, 277,  90, 294, 461, 342, 144, 509, 130,  50, 478, 464, 535, 621,
        293, 575, 591, 568,  25, 743, 368, 486, 414, 329, 178, 557, 583, 302,
        343, 171, 604,  87, 505,  45, 567, 289, 128, 347,  66, 608, 536, 282,
        176, 451, 696, 104, 613, 494, 267, 310, 618, 636, 213,   3, 435, 440,
        353,  21])


In [3]:
model = torchreid.models.build_model(
    name=model_name,
    num_classes=datamanager.num_train_pids, #2000 for Tango
    loss="triplet",
    pretrained=True,
    pooling='gem',
    # custom_layers=['ArcFaceMarginProduct']
)
model = model.cuda()
optimizer = torchreid.optim.build_optimizer(
    model,
    optim="adam",
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler="single_step",
    stepsize=20
)

In [4]:
# from tools.visualize_actmap import visactmap
# test_loader = datamanager.test_loader
# visactmap(model, test_loader, f"log/visactmap_{model_name}_{dataset_name}", 128, 256, True, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

In [5]:
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True
)

In [6]:
engine.run(
    save_dir=f"log/{model_name}_{dataset_name}",
    max_epoch=60,
    eval_freq=5,
    print_freq=50,
    test_only=False,
    visrank=False,
    eval_metric='default'
)

=> Start training
epoch: [1/60][50/101]	time 0.152 (0.167)	data 0.000 (0.004)	eta 0:16:41	loss 6.6207 (7.5141)	acc 12.5000 (3.0312)	lr 0.000300
epoch: [1/60][100/101]	time 0.150 (0.159)	data 0.000 (0.002)	eta 0:15:46	loss 5.1849 (6.6385)	acc 24.2188 (11.7969)	lr 0.000300
epoch: [2/60][50/101]	time 0.150 (0.154)	data 0.000 (0.004)	eta 0:15:11	loss 3.7649 (4.0638)	acc 48.4375 (49.9844)	lr 0.000300
epoch: [2/60][100/101]	time 0.150 (0.152)	data 0.000 (0.002)	eta 0:14:52	loss 2.9943 (3.7133)	acc 76.5625 (57.5625)	lr 0.000300
epoch: [3/60][50/101]	time 0.151 (0.154)	data 0.000 (0.004)	eta 0:14:55	loss 2.6109 (2.7483)	acc 82.0312 (80.7500)	lr 0.000300
epoch: [3/60][100/101]	time 0.150 (0.152)	data 0.000 (0.002)	eta 0:14:36	loss 2.5263 (2.6865)	acc 87.5000 (82.2500)	lr 0.000300
epoch: [4/60][50/101]	time 0.150 (0.154)	data 0.000 (0.004)	eta 0:14:38	loss 2.3283 (2.3886)	acc 91.4062 (90.2188)	lr 0.000300
epoch: [4/60][100/101]	time 0.151 (0.152)	data 0.000 (0.002)	eta 0:14:21	loss 2.3711 (2.371

Exception ignored in: <function _releaseLock at 0x77676b0839a0>
Traceback (most recent call last):
  File "/home/trewq/anaconda3/envs/torchreid/lib/python3.10/logging/__init__.py", line 228, in _releaseLock
    def _releaseLock():
KeyboardInterrupt: 


Done, obtained 3368-by-2048 matrix
Extracting features from gallery set ...
Done, obtained 15913-by-2048 matrix
Speed: 0.0018 sec/batch
Computing distance matrix with metric=euclidean ...
Computing CMC and mAP ...
[   0  455    0 ... 1026  668  723] <class 'numpy.ndarray'> False
